In [ ]:
from headers.directory_manager import *

ROOT_KAIST = "/home/dimitrios/THESIS_DATASETS/kaist_original"

BASELINE_MODEL = "./configs/yolo11m.pt" # OR SOME OTHER BASELINE MODEL (YOLOv11 family model)
FINE_TUNED_IR_YOLO_MODEL = "FINE_TUNED_MODEL"

nighttime_sequences = ["03", "04", "05", "10", "11"]


# flatten lwir visible and labels for nighttime sequences


In [ ]:
flattened_nighttime_lwir_images = flatten_KAIST_images(sequences= nighttime_sequences, 
                                                       root_kaist= ROOT_KAIST, 
                                                       target_directory="preprocessed_nighttime", 
                                                       image_type= "lwir")

flattened_nighttime_rgb_images = flatten_KAIST_images(sequences= nighttime_sequences, 
                                                       root_kaist= ROOT_KAIST, 
                                                       target_directory="preprocessed_nighttime", 
                                                       image_type= "visible")

flattened_nighttime_labels = flatten_KAIST_labels(sequences= nighttime_sequences, 
                                                  root_kaist= ROOT_KAIST, 
                                                  target_directory="preprocessed_nighttime")

# generate config file containing only pedestrians and and empty frames 

In [ ]:
nighttime_pedestrian_and_empty_image_ids = "./configs/nighttime_ped_emp_only.txt"

generate_txt_config_by_components(label_dir     = flattened_nighttime_labels,
                                strict          = True,
                                include_person  = True, 
                                include_people  = False, 
                                include_empty   = True, 
                                include_cyclist = False, 
                                output_file= nighttime_pedestrian_and_empty_image_ids)

In [ ]:
cleaned_nighttime_labels_ped_emp = flattened_nighttime_labels + "ped_emp"

copy_files_based_on_config(input_dir=flattened_nighttime_labels, 
                           output_dir=cleaned_nighttime_labels_ped_emp, 
                           config_file=nighttime_pedestrian_and_empty_image_ids)

yolo_nighttime_labels = convert_kaist_to_yolo_labels(source_directory= cleaned_nighttime_labels_ped_emp)

# convert test sequence to YOLO format

In [ ]:
test_sequence_lables = os.path.join(ROOT_KAIST, "annotations-xml-new-sanitized/set09/V000")
test_sequence_images = os.path.join(ROOT_KAIST, "images/set09/V000/visible")

In [ ]:
test_sequence_pedestrians_and_empty = "./configs/nighttime_test_sequence_ped_emp_only.txt"

generate_txt_config_by_components(label_dir     = test_sequence_lables,
                                strict          = True,
                                include_person  = True, 
                                include_people  = False, 
                                include_empty   = True, 
                                include_cyclist = False, 
                                output_file= test_sequence_pedestrians_and_empty)

In [ ]:
cleaned_up_test_sequence_labels = os.path.join(ROOT_KAIST, "cleaned_up_test_labels")
copy_files_based_on_config(input_dir=test_sequence_lables, 
                           output_dir=cleaned_up_test_sequence_labels, 
                           config_file=test_sequence_pedestrians_and_empty)


cleaned_up_test_sequence_images = os.path.join(ROOT_KAIST, "cleaned_up_test_images")
copy_files_based_on_config(input_dir=test_sequence_images, 
                           output_dir=cleaned_up_test_sequence_images, 
                           config_file=test_sequence_pedestrians_and_empty)



In [ ]:
test_sequence_labels_cleaned = convert_kaist_to_yolo_labels(source_directory= test_sequence_lables)


In [ ]:
cleaned_nighttime_rgb_ped_emp = flattened_nighttime_rgb_images + "ped_emp"

copy_files_based_on_config(input_dir=flattened_nighttime_rgb_images, 
                           output_dir=cleaned_nighttime_rgb_ped_emp, 
                           config_file=nighttime_pedestrian_and_empty_image_ids)

In [ ]:
cleaned_nighttime_lwir_ped_emp = flattened_nighttime_lwir_images + "ped_emp"

copy_files_based_on_config(input_dir=flattened_nighttime_lwir_images, 
                           output_dir=cleaned_nighttime_lwir_ped_emp, 
                           config_file=nighttime_pedestrian_and_empty_image_ids)

In [ ]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

In [ ]:
model = YOLO(FINE_TUNED_IR_YOLO_MODEL)
lwir_generated_yolo_labels = generate_yolo_labels(model=model, image_dir=cleaned_nighttime_lwir_ped_emp)

# generated labels for nighttime section of the dataset using the fine tuned IR-YOLO model 

In [ ]:
dataset_generated_labels = {
    "train":{
        "images": cleaned_nighttime_rgb_ped_emp,
        "labels": lwir_generated_yolo_labels,
        },
    "test":{
        "images": cleaned_up_test_sequence_images,
        "labels": test_sequence_labels_cleaned
    }
}
generated_labels_tts = os.path.join(ROOT_KAIST, "tts_generated_labels_ir-yolov11m")

organize_dataset(root_path=generated_labels_tts, dataset_dict=dataset_generated_labels)

In [ ]:
yaml_ir_yolo_training = {
    "train": os.path.join(generated_labels_tts, "train"),
    "val":   os.path.join(generated_labels_tts, "test"),
    "nc": 1,
    "names": ['person']  
}

yaml_daytime_labels = "./configs/ir-yolov11m_tts.yaml"

write_yolo_yaml(config_dict= yaml_ir_yolo_training, output_path=yaml_daytime_labels)

In [ ]:
from ultralytics import YOLO
import subprocess
import torch

torch.cuda.empty_cache()
try:
    subprocess.check_output('nvidia-smi')
    print('Nvidia GPU detected!')
except Exception: # this command not being found can raise quite a few different errors depending on the configuration
    print('No Nvidia GPU in system!')
    

torch.cuda.is_available()

# training of a baseline model for low-light pedestrian detection with generated labels

In [ ]:
BASELINE_MODEL = "./configs/yolo11m.pt"
yaml_daytime_labels = "./configs/ir-yolov11m_tts.yaml"

model = YOLO(BASELINE_MODEL)  
results = model.train(model = model, data=yaml_daytime_labels, epochs=10, device=0, batch=8)

# data preprocessing for nighttime KAIST labels

the process is the same as before with the key difference of having the tran labels be the ones provided by KAIST

In [ ]:
dataset_generated_labels = {
    "train":{
        "images": cleaned_nighttime_rgb_ped_emp,
        "labels": yolo_nighttime_labels,
        },
    "test":{
        "images": cleaned_up_test_sequence_images,
        "labels": test_sequence_labels_cleaned
    }
}

generated_labels_tts = os.path.join(ROOT_KAIST, "tts_KAIST_labels")

organize_dataset(root_path=generated_labels_tts, dataset_dict=dataset_generated_labels)

In [ ]:
yaml_kaist_training = {
    "train": os.path.join(generated_labels_tts, "train"),
    "val":   os.path.join(generated_labels_tts, "test"),
    "nc": 1,
    "names": ['person']  
}

yaml_nighttime_labels_kaist = "./configs/kaist_labels_tts.yaml"

write_yolo_yaml(config_dict= yaml_kaist_training, output_path=yaml_nighttime_labels_kaist)

In [ ]:
model = YOLO(BASELINE_MODEL)  
results = model.train(data=yaml_nighttime_labels_kaist, epochs=10, device=0, batch=8)